### Wave equation in two dimensions
Our goal is to compute and visualize solutions $u(x,y,t)$ of the wave equation $$u_{tt}=c^2 \Delta u$$ with $\Delta u =u_{xx}+u_{yy}$ on the square $(x,y)\in[0,L]\times[0,L]$ with initial conditions $u(x,y,0)=f(x,y)$ and $u_t(x,y,0)=g(x,y)$. Our code will allow us to use Dirichlet, Neumann, or periodic boundary conditions. [Linked here is a movie of light rain falling in a pond](https://youtu.be/d1GSHAMpNYo). We first load the necessary Python modules.

In [4]:
%matplotlib widget
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as colors
from matplotlib.colors import LightSource

Next, we define the following functions:
* `discrete_laplacian` provides a centered finite-difference approximation of the Laplace operator $\Delta u =u_{xx}+u_{yy}$;
* `leapfrog` implements the leapfrog method to evolve the solution in time. Leapfrog takes advantage of the fact that the wave equation contains second-order derivatives in time; 
* `define_initial_condition` uses the given initial conditions to compute the first two solution iterates;
* `update_solution` uses the leapfrog method applied to the time derivative $u_{tt}$ to update the solution to the next time step;
* `update_graph` animates the solution.

In [5]:
def discrete_laplacian(u, bdy):
    if bdy == 'Periodic':
        L = -4*u
        L += np.roll(u, (0,-1), (0,1))
        L += np.roll(u, (0,+1), (0,1))
        L += np.roll(u, (-1,0), (0,1))
        L += np.roll(u, (+1,0), (0,1))
        return L
    elif bdy == 'Dirichlet':
        v = np.pad(u, 1, constant_values=0)
    elif bdy == 'Neumann':
        v = np.pad(u, 1, mode='edge')
    L = -4*v
    L += np.roll(v, (0,-1), (0,1))
    L += np.roll(v, (0,+1), (0,1))
    L += np.roll(v, (-1,0), (0,1))
    L += np.roll(v, (+1,0), (0,1))
    L = L[1:-1,1:-1]
    return L

def leapfrog(u0, um, cfl, bdy):
    up = 2*u0 - um + cfl**2*discrete_laplacian(u0, bdy)
    return up, u0

def define_initial_condition(f, g, dt, bdy):
    u0 = f + 0.5*cfl**2*discrete_laplacian(f, bdy) + dt*g
    return u0, f

def update_solution(f, g, cfl, dt, bdy, Nframes, Nskip):
    n = 0
    u0, um = define_initial_condition(f, g, dt, bdy)
    while n<Nframes:
        n += 1
        for k in range(Nskip):
            u0, um = leapfrog(u0, um, cfl, bdy)
        yield u0

def update_graph(u, ls, imu):
    imu.set_array(ls.hillshade(u))

In the next cell, we define the speed constant $c>0$, set the initial conditions $u(x,0)=f(x)$ and $u_t(x,0)=g(x)$, and run the animation of the solution.

In [15]:
# model parameter
c = 1       # speed constant
L = 30      # domain length

# numerical parameters
N = 800    # grid size

# define a function that simulates a raindrop
def pebble(X, Y, width, positionX, positionY):
    r = width*np.sqrt((X-positionX)**2 + (Y-positionY)**2)
    u = np.random.uniform(-1., 1., 1)*np.cos(1.9*r)/np.cosh(r)
    return u

x = np.linspace(0, L, N)
X, Y = np.meshgrid(x, x)

# initial displacement consists of three separate raindrops 
f =  pebble(X, Y, 3.5, 0.35*L, 0.5*L)
f += pebble(X, Y, 3, 0.6*L, 0.6*L)
f += pebble(X, Y, 4, 0.7*L, 0.2*L)

# initial velocity is zero
g = np.zeros((N,N))

# animation parameters
Nsteps = 400
Nskip = 10

# set time step to satisfy leapfrog stability criterion (cfl < 1/sqrt(2) ~ 0.7)
cfl = 0.6
dx = L/N
dt = cfl * dx / c

# compute and animate solution
Nframes = int(Nsteps/Nskip)
extent = [0, L, 0, L]

fig, ax = plt.subplots(1,1,figsize=(5,5))
ls = LightSource(azdeg=220, altdeg=70)
imu = ax.imshow(ls.hillshade(f), cmap='Greys', animated=True, extent=extent, origin='lower')
ani = animation.FuncAnimation(fig, update_graph,
                              update_solution(f, g, cfl, dt, 'Neumann', Nframes, Nskip),
                              fargs=(ls, imu), repeat=False)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …